# Data reduction for POWGEN

This notebook shows a basic reduction workflow for powder diffraction for the SNS [POWGEN](https://sns.gov/powgen) instrument.
It serves mainly to develop and present routines for powder diffraction and will eventually be removed in favor of a workflow for DREAM at ESS.

First, we import all required modules.
Note that we specifically import the `diffraction` and `powgen` submodules of `ess`.

In [ ]:
import scipp as sc
import scippneutron as scn

import ess
from ess import diffraction
from ess.external import powgen

Initialize the logging system to write logs to a file `powgen.log`.
This also displays a widget which shows log messages emitted by the functions that we call.

In [ ]:
ess.logging.configure_workflow('powgen_reduction', filename='powgen.log')

## Load data

Load the sample data.
We use the dedicated `load` function from `powgen` instead of `scippneutron.load` because the former passes the correct parameters to the underlying algorithm automatically.

**Note:** We get the file name from `powgen.data`.
This module provides access to managed example files.
In the real world, we would need to find the file name in a different way.

In [ ]:
sample = powgen.load(powgen.data.sample_file())

In [ ]:
sample

## Inspect the raw data

We can plot the data array to get an idea of its contents.

In [ ]:
sample.plot()

We can see how that data maps onto the detector by using scippneutron's instrument view.

In [ ]:
scn.instrument_view(sample,
                    camera={'position': sc.vector(value=[-3, 3, 3],
                                                  unit=sample.coords['position'].unit)})

## Filter out invalid events

The file contains events that cannot be associated with a specific pulse.
We can get a range of valid time-of-flight values from the instrument characterization file associated with the current run.
There is currently no mechanism in `scippneutron` or `ess` to load such a file as it is not clear if ESS will use this approach.
The values used below are taken from `PG3_characterization_2011_08_31-HR.txt` which is part of the sample files of Mantid.
See, e.g., [PowderDiffractionReduction](https://www.mantidproject.org/PowderDiffractionReduction).

We remove all events that have a time-of-flight value outside the valid range:

In [ ]:
sample = sc.bin(sample, edges=[
    sc.array(dims=['tof'], values=[0.0, 16666.67], unit='us')
])

## Normalize by proton charge

Next, we normalize the data by the proton charge.

In [ ]:
diffraction.normalize_by_proton_charge(sample, in_place=True)

We can check the unit of the event weight to see that the data was indeed divided by a charge.

In [ ]:
sample.data.values[0].unit

## Compute d-spacing

Here, we compute d-spacing using calibration parameters provided in an example file.

First, we load the calibration parameters.
We need to specify an instrument definition that Mantid understands.
It needs to be selected for the specific data that we are processing.

**Note:** ESS instruments will not use instrument definition files (IDF).
Instead, the instrument parameters will be encoded in the `ess` module and the instrument can be identified using data in the NeXus file.
But for the purposes of this example, we need to provide the IDF.

In [ ]:
cal = diffraction.load_calibration(
    powgen.data.calibration_file(),
    instrument_filename='POWGEN_Definition_2011-02-25.xml',
)

The calibration is loaded with a 'detector' dimension.
Compute the corresponding spectrum indices using the detector info loaded as part of the sample data.

In [ ]:
cal = powgen.beamline.map_detector_to_spectrum(
    cal, detector_info=sample.coords['detector_info'].value)

In [ ]:
cal

Now when can compute d-spacing for the sample using the calibration parameters.

In [ ]:
sample_dspacing = diffraction.to_dspacing_with_calibration(sample,
                                                           calibration=cal)

## Vanadium correction

Before we can process the d-spacing distribution further, we need to normalize the data by a vanadium measurement.
`ess.diffraction` provides a helper function to load event data for vanadium and process it in a similar way to what we did above.

In [ ]:
vana = diffraction.load_and_preprocess_vanadium(
    powgen.data.vanadium_file(), powgen.data.empty_instrument_file())

In [ ]:
vana

Have a look at the log widget at the top of the notebook.
There is a message showing that the empty instrument data was not used because its proton charge is too low.
That is, it contains effectively no valid events.
This is a crude, preliminary event filtering mechanism applied by `load_and_preprocess_vanadium`.

Now, we compute d-spacing using the same calibration parameters as before.

In [ ]:
vana_dspacing = diffraction.to_dspacing_with_calibration(vana, calibration=cal)

In [ ]:
vana_dspacing

## Inspect d-spacing

We need to histogram the events in order to normalize our sample data by vanadium.
For consistency, we use these bin edges for both vanadium and the sample data.

In [ ]:
d = vana_dspacing.coords['dspacing']
dspacing_edges = sc.linspace('dspacing',
                             d.min().value,
                             d.max().value,
                             200,
                             unit=d.unit)

### All spectra combined

We start simple by combining all spectra using `data.bins.concat('spectrum')`.
Then, we can normalize the same data by vanadium to get a d-spacing distribution.

In [ ]:
all_spectra = diffraction.normalize_by_vanadium(
    sample_dspacing.bins.concat('spectrum'),
    vanadium=vana_dspacing.bins.concat('spectrum'),
    edges=dspacing_edges,
)

In [ ]:
all_spectra.plot()

### Focus into $2\theta$ bins

For a better resolution, we now group ('focus') the sample and vanadium data into a number of bins in the scattering angle [$2\theta$](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
and normalize each individually.

In [ ]:
two_theta = sc.linspace(dim='two_theta',
                        unit='deg',
                        start=25.0,
                        stop=90.0,
                        num=16)
sample_focussed = diffraction.focus_by_two_theta(sample_dspacing,
                                                 edges=two_theta)
vana_focussed = diffraction.focus_by_two_theta(vana_dspacing, edges=two_theta)

In [ ]:
normalized = diffraction.normalize_by_vanadium(sample_focussed,
                                               vanadium=vana_focussed,
                                               edges=dspacing_edges)

Now we can inspect the d-spacing distribution as a function of $2\theta$.

In [ ]:
normalized.plot()

In order to get 1-dimensional plots, we can select some ranges of scattering angles.

In [ ]:
angle = sc.midpoints(normalized.coords['two_theta']).values
results = {
    f'{round(angle[group], 3)} rad':
    normalized['two_theta', group]
    for group in range(2, 6)
}
sc.plot(results)

Or interactively by plotting with a 1d projection.

In [ ]:
normalized.plot(projection='1d')